<a href="https://colab.research.google.com/github/thiagocunha88/DataScience-Challenge_DS_Alura/blob/master/Challenge_Alura_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge Alura - Reduzir a taxa de evasão dos clientes (Churn Rate)
## Analisar os clientes para identificar um padrão que determine aqueles que possuem maior chance de deixar de consumir o serviço, ou deixar de ser cliente da empresa

## Para iniciar os estudos, inicia-se a importação das bibliotecas básicas para tal e o dataset a ser analisado, entendendo que o primeiro passo é entender os dados a serem utilizados.

### Importando bibliotecas

Para iniciar uma análise exploratória é necessário transformar e visualizar os dados.

Foram selecionadas duas bibliotecas muito comuns e de bom desempenho para isto, o pandas e numpy.

Posteriormente serão importadas outras bibliotecas, principalmente quando forem geradas visualizações gráficas, mas isso pode aguardar.

In [18]:
import pandas as pd
import numpy as np

pd.options.plotting.backend = 'plotly'

### Importando o dataset e tendo a primeira visualização

Como a base dados a ser estudada está em formato json, optei por utilizar o pd.read_json() para fazer a leitura, principalmente porque apresenta os dados em formato tabular, facilitando as primeiras impressões e possibilitando nortear os próximos passos.

In [19]:
df = pd.read_json('https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json')
display(df.head(2))

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Os dados importados geraram uma tabela, porém, a mesma apresenta colunas ainda em formato json, dificultando o entendimento e impossibilitando análises estatísticas básicas de maneira simples.

Sendo assim, será necessário iniciar a transformação dos dados.

### Transformando os dados de json para dataframe pandas.

Todas as colunas a partir da Churn estavam em formato json ainda, dificultando a visualização completa bem como cálculos e análises a serem realizados.

Foi efetuada a conversão dos dados, primeiramente, criando dataframes separados para cada coluna a ser tratada (customer, phone, internet e account), utilizando a função pd.json_normalize().

Na coluna account foi necessário uma atenção maior, já que havia uma cadeia de json (Charges), situação que foi tratada pela função pd.json_normalize().

In [20]:
# Separando valores que estão em formato json, e retornando para o dataframe
customer = pd.json_normalize(df.customer)
phone = pd.json_normalize(df.phone)
internet = pd.json_normalize(df.internet)
account = pd.json_normalize(df.account)
df_base = pd.concat([df.customerID, df.Churn, customer, phone, internet, account], axis = 1)

### Fazendo a tradução dos cabeçalhos

A base de dados está toda em inglês, e para facilitar o entendimento do que cada coluna significa, foi efetuada a tradução das mesmas.

Posteriormente será analisada a necessidade de traduzir também os outros dados do dataset.

In [21]:
# Traduzindo nomes das colunas manualmente
df_base.columns = ['clienteID', 
                   'Churn', 
                   'genero_cliente', 
                   'mais_que_65_anos', 
                   'possui_parceiro_a', 
                  'possui_dependentes', 
                  'meses_contrato',
                  'servico_telefonico',
                  'multiplas_linhas',
                  'servico_internet',
                  'assinatura_seguranca',
                  'assinatura_backup',
                  'assinatura_protecao_dispositivo',
                  'assinatura_suporte',
                  'assinatura_TV',
                  'assinatura_streaming',
                  'contrato',
                  'fatura_digital',
                  'forma_pagamento',
                  'total_servicos_mes',
                  'total_gasto']

df_base.head()

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


Agora que foi possível entender do que se trata cada coluna é importante verificar se os dados existentes nelas são consistentes e possibilitam efetuar análises.

Primeiramente é necessário verificar se há dados faltantes, e se eles impossibilitarão análises posterior.

O pandas tem a função .info() que retorna os tipos de cada coluna, além de informar se há dados nulos nelas.

In [22]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   clienteID                        7267 non-null   object 
 1   Churn                            7267 non-null   object 
 2   genero_cliente                   7267 non-null   object 
 3   mais_que_65_anos                 7267 non-null   int64  
 4   possui_parceiro_a                7267 non-null   object 
 5   possui_dependentes               7267 non-null   object 
 6   meses_contrato                   7267 non-null   int64  
 7   servico_telefonico               7267 non-null   object 
 8   multiplas_linhas                 7267 non-null   object 
 9   servico_internet                 7267 non-null   object 
 10  assinatura_seguranca             7267 non-null   object 
 11  assinatura_backup                7267 non-null   object 
 12  assinatura_protecao_

A coluna total_gasto apresenta o tipo 'object', porém, se trata de um valor numérico flutuante, ou float, e por isto se faz necessária a conversão do mesmo.

Utilizei a função pd.to_numeric para isto.

In [23]:
# Convertendo coluna total_gasto para float
df_base.total_gasto = pd.to_numeric(df_base.total_gasto, errors='coerce')
df_base.head(2)

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4


Após converter os dados, importante verificar os dados obtidos, pois na função to_numeric foi utilizado o argumento *errors = 'coerce'*. Este argumento faz com que os dados que tenham erros sejam covertidos em NaN, e estes podem atrapalhar a análise e montagem de gráficos.

In [24]:
df_base[df_base.total_gasto.isna()]

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,...,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,...,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,...,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,...,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN


Foram identificadas 11 linhas em que o total_gasto contém NaN, coincidindo com as linhas em que os clientes ainda não possuem um mês completo de utilização dos serviços (coluna meses_contrato = 0).

Será considerado o valor da coluna total_servicos_mes nestes casos, pois indica o valor gasto pelo cliente em todo o período de assinatura do mesmo (no caso, um mês incompleto).

In [25]:
df_base.loc[df_base.total_gasto.isna(), 'total_gasto'] = df_base.loc[df_base.total_gasto.isna(), 'total_servicos_mes']
df_base[df_base.meses_contrato == 0][['total_servicos_mes', 'total_gasto']]

,total_servicos_mes,total_gasto
975,56.05,56.05
1775,20.00,20.00
1955,61.90,61.90
2075,19.70,19.70
2232,20.25,20.25
2308,25.35,25.35
2930,73.35,73.35
3134,25.75,25.75
3203,52.55,52.55
4169,80.85,80.85


Achei interessante a ideia de criar uma função para validar os valores apresentados nas colunas para evitar a repetição de código.

Aproveitei pra criar uma lista de todos os valores únicos, para possibilitar uma tradução dos valores contidos na tabela, além dos valores das colunas (que já foram traduzidos).

In [26]:
df_base.loc[:,'Churn':].columns

Index(['Churn', 'genero_cliente', 'mais_que_65_anos', 'possui_parceiro_a',
       'possui_dependentes', 'meses_contrato', 'servico_telefonico',
       'multiplas_linhas', 'servico_internet', 'assinatura_seguranca',
       'assinatura_backup', 'assinatura_protecao_dispositivo',
       'assinatura_suporte', 'assinatura_TV', 'assinatura_streaming',
       'contrato', 'fatura_digital', 'forma_pagamento', 'total_servicos_mes',
       'total_gasto'],
      dtype='object')

In [27]:
def valores_unicos(df):
    lista_valores_unicos = []
    for coluna in df.loc[:,'Churn':].columns:
        if len(df[f'{coluna}'].unique()) > 5:
            print(f'{coluna} contém muitos valores únicos, analisar com maior detalhamento.')
        else:
            valores = df[f'{coluna}'].unique()
            lista_valores_unicos.extend(list(valores))
            print(f'{coluna}: ' + str(valores))
    lista_valores_unicos = list(set(lista_valores_unicos))
    return lista_valores_unicos

In [28]:
lista_valores_unicos = valores_unicos(df_base)

Churn: ['No' 'Yes' '']
genero_cliente: ['Female' 'Male']
mais_que_65_anos: [0 1]
possui_parceiro_a: ['Yes' 'No']
possui_dependentes: ['Yes' 'No']
meses_contrato contém muitos valores únicos, analisar com maior detalhamento.
servico_telefonico: ['Yes' 'No']
multiplas_linhas: ['No' 'Yes' 'No phone service']
servico_internet: ['DSL' 'Fiber optic' 'No']
assinatura_seguranca: ['No' 'Yes' 'No internet service']
assinatura_backup: ['Yes' 'No' 'No internet service']
assinatura_protecao_dispositivo: ['No' 'Yes' 'No internet service']
assinatura_suporte: ['Yes' 'No' 'No internet service']
assinatura_TV: ['Yes' 'No' 'No internet service']
assinatura_streaming: ['No' 'Yes' 'No internet service']
contrato: ['One year' 'Month-to-month' 'Two year']
fatura_digital: ['Yes' 'No']
forma_pagamento: ['Mailed check' 'Electronic check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
total_servicos_mes contém muitos valores únicos, analisar com maior detalhamento.
total_gasto contém muitos valores úni

In [29]:
lista_valores_unicos = np.sort(lista_valores_unicos)
lista_valores_unicos

array(['', '0', '1', 'Bank transfer (automatic)',
       'Credit card (automatic)', 'DSL', 'Electronic check', 'Female',
       'Fiber optic', 'Mailed check', 'Male', 'Month-to-month', 'No',
       'No internet service', 'No phone service', 'One year', 'Two year',
       'Yes'], dtype='<U25')

Com a lista de valores únicos pronta, basta criar uma lista com os valores traduzidos e executar um .replace no dataframe inteiro.

Apenas para manter o padrão, criei novo dataframe inidicando que se trata de valores traduzidos.

In [30]:
lista_valores_unicos_traduzidos = [
                                   'Transferência bancária (automática)',
                                   'Cartão de crédito (automático)',
                                   'DSL',
                                   'Cheque eletrônico',
                                   'Feminino',
                                   'Fibra ótica',
                                   'Cheque por correio',
                                   'Masculino',
                                   'Mês a mês',
                                   'Não',
                                   'Sem serviço de internet',
                                   'Sem serviço telefônico',
                                   'Um ano',
                                   'Dois anos',
                                   'Sim'
                                   ]

df_base_traduzido = df_base.replace(lista_valores_unicos[3:], lista_valores_unicos_traduzidos)
df_base_traduzido.sample(5)

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
194,0305-SQECB,Não,Feminino,0,Não,Sim,11,Não,Sem serviço telefônico,DSL,...,Não,Não,Sim,Não,Não,Um ano,Sim,Cheque por correio,36.05,402.60
3524,4854-SSLTN,Não,Masculino,0,Sim,Sim,2,Sim,Não,DSL,...,Sim,Não,Sim,Não,Não,Dois anos,Não,Cheque por correio,59.50,130.50
819,1163-ONYEY,Não,Masculino,0,Sim,Sim,41,Sim,Sim,DSL,...,Sim,Sim,Não,Sim,Não,Mês a mês,Sim,Cartão de crédito (automático),74.55,3023.55
1329,1925-GMVBW,Não,Feminino,0,Não,Não,5,Sim,Não,Não,...,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Mês a mês,Não,Cartão de crédito (automático),20.55,96.10
1603,2273-QCKXA,Não,Masculino,0,Não,Não,1,Sim,Não,DSL,...,Não,Não,Sim,Não,Não,Mês a mês,Não,Cheque por correio,49.05,49.05


In [31]:
valores_unicos(df_base_traduzido)

Churn: ['Não' 'Sim' '']
genero_cliente: ['Feminino' 'Masculino']
mais_que_65_anos: [0 1]
possui_parceiro_a: ['Sim' 'Não']
possui_dependentes: ['Sim' 'Não']
meses_contrato contém muitos valores únicos, analisar com maior detalhamento.
servico_telefonico: ['Sim' 'Não']
multiplas_linhas: ['Não' 'Sim' 'Sem serviço telefônico']
servico_internet: ['DSL' 'Fibra ótica' 'Não']
assinatura_seguranca: ['Não' 'Sim' 'Sem serviço de internet']
assinatura_backup: ['Sim' 'Não' 'Sem serviço de internet']
assinatura_protecao_dispositivo: ['Não' 'Sim' 'Sem serviço de internet']
assinatura_suporte: ['Sim' 'Não' 'Sem serviço de internet']
assinatura_TV: ['Sim' 'Não' 'Sem serviço de internet']
assinatura_streaming: ['Não' 'Sim' 'Sem serviço de internet']
contrato: ['Um ano' 'Mês a mês' 'Dois anos']
fatura_digital: ['Sim' 'Não']
forma_pagamento: ['Cheque por correio' 'Cheque eletrônico' 'Cartão de crédito (automático)'
 'Transferência bancária (automática)']
total_servicos_mes contém muitos valores únicos, an

['',
 0,
 1,
 'Masculino',
 'Sem serviço telefônico',
 'Fibra ótica',
 'Cartão de crédito (automático)',
 'Sem serviço de internet',
 'Feminino',
 'Um ano',
 'Cheque por correio',
 'Não',
 'DSL',
 'Dois anos',
 'Transferência bancária (automática)',
 'Mês a mês',
 'Cheque eletrônico',
 'Sim']

In [32]:
display((df_base_traduzido.groupby('Churn').count()['clienteID']))
display((df_base_traduzido.groupby('Churn').count()['clienteID'] / df_base_traduzido.Churn.count() * 100))


Churn
        224
Não    5174
Sim    1869
Name: clienteID, dtype: int64

Churn
        3.082427
Não    71.198569
Sim    25.719004
Name: clienteID, dtype: float64

Com a informação dos valores únicos, foi possível perceber que a coluna Churn possui, além de Yes e No, valores vazios. Para ser mais preciso, 224 ocorrências, ou 3,08%.

O foco do estudo é endenter o comportamento de clientes usando como variável dependente o Churn, além do que, a quantidade de vazios não é expressiva e por isto não vejo sentido em manter as linhas em que este valore está vazio.

Para fazer isto, utilizei a função .drop e .reset_index.

In [33]:
df_base_traduzido.drop(df_base[df_base.Churn == ''].index, inplace = True)
df_base_traduzido.reset_index(drop = True)
df_base_traduzido.sample(5)

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_backup,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto
1710,2430-RRYUW,Não,Masculino,0,Não,Não,66,Sim,Não,DSL,...,Sim,Não,Não,Não,Não,Um ano,Sim,Cheque por correio,54.80,3465.70
5000,6840-RESVB,Não,Masculino,0,Sim,Sim,24,Sim,Sim,DSL,...,Não,Sim,Sim,Sim,Sim,Um ano,Sim,Cheque por correio,84.80,1990.50
5136,7013-PSXHK,Não,Feminino,0,Não,Não,40,Sim,Sim,DSL,...,Não,Sim,Sim,Sim,Sim,Um ano,Sim,Cheque por correio,80.75,3208.65
2526,3523-QRQLL,Sim,Feminino,0,Sim,Sim,22,Sim,Não,DSL,...,Sim,Não,Não,Sim,Sim,Um ano,Sim,Transferência bancária (automática),69.50,1498.20
5067,6921-OZMFH,Não,Masculino,0,Sim,Sim,5,Sim,Sim,DSL,...,Não,Não,Sim,Não,Não,Mês a mês,Sim,Cheque por correio,55.70,259.40


Outro dado interessante para ter no dataframe é o gasto diário médio.

Achei interessante validar se o total_servicos_mes coincide com o total_gasto dividido por meses_contrato, pois assim será possível escolher qual o melhor valor para calcular o gasto diário.

In [39]:
df_base_traduzido['teste'] = (df_base_traduzido.total_gasto / df_base_traduzido.meses_contrato) - df_base_traduzido.total_servicos_mes
df_base_traduzido.teste.plot(kind='hist')



Para criar essa coluna, fiz uma *list comprehension*, validando a coluna meses_contrato. Caso esta coluna possua valor maior que 0, basta dividir o total_gasto por meses_contrato vezes 30, do contrário apenas dividir total_gasto por 30.

Poderia usar o total_servicos_mes e dividir por 30, mas

In [35]:
df_base_traduzido['gastos_diarios'] = [df_base_traduzido.total_gasto[i] / (df_base_traduzido.meses_contrato[i] * 30) 
                             if df_base_traduzido.meses_contrato[i] > 0 
                             else df_base_traduzido.total_gasto[i] / 30 
                             for i in df_base_traduzido.index]

df_base_traduzido.sample(5)

,clienteID,Churn,genero_cliente,mais_que_65_anos,possui_parceiro_a,possui_dependentes,meses_contrato,servico_telefonico,multiplas_linhas,servico_internet,...,assinatura_protecao_dispositivo,assinatura_suporte,assinatura_TV,assinatura_streaming,contrato,fatura_digital,forma_pagamento,total_servicos_mes,total_gasto,gastos_diarios
4474,6131-JLWZM,Não,Feminino,0,Não,Não,13,Sim,Não,Não,...,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Mês a mês,Sim,Cheque por correio,20.30,275.40,0.706154
5762,7873-CVMAW,Não,Masculino,0,Não,Não,72,Sim,Sim,DSL,...,Sim,Sim,Sim,Sim,Dois anos,Não,Cartão de crédito (automático),88.55,6362.35,2.945532
5981,8174-TBVCF,Não,Feminino,0,Sim,Não,70,Sim,Sim,DSL,...,Sim,Sim,Sim,Sim,Dois anos,Sim,Cartão de crédito (automático),94.80,6859.05,3.266214
3462,4800-VHZKI,Sim,Feminino,0,Não,Não,1,Sim,Não,Não,...,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Sem serviço de internet,Mês a mês,Sim,Cheque por correio,19.90,19.90,0.663333
4668,6390-DSAZX,Sim,Feminino,0,Não,Sim,1,Sim,Não,Fibra ótica,...,Não,Não,Não,Não,Mês a mês,Sim,Cheque por correio,73.60,73.60,2.453333


### Validação de algumas colunas

In [ ]:
df_base.columns

In [ ]:
df_base.assinatura_protecao_dispositivo.unique()

In [ ]:
df_base[df_base.multiplas_linhas == 'No phone service'].shape[0]

In [ ]:
pd.crosstab(df_base.Churn, df_base.servico_internet)

In [ ]:
df_base.groupby('servico_internet').count()

In [ ]:
df_base.groupby('assinatura_seguranca').count()

In [ ]:
df_base.groupby('assinatura_backup').count()

In [ ]:
df_base.groupby('assinatura_protecao_dispositivo').count()

### Análise da variável target - Churn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_base.Churn.value_counts()

#### Avaliando por gênero

In [ ]:
df_genero = pd.crosstab(df_base.Churn, df_base.genero_cliente)
df_genero

In [ ]:
df_base.groupby('genero_cliente')['Churn'].count()

In [ ]:
df_genero = pd.crosstab(df_base.Churn, df_base.genero_cliente)

fig, ax = plt.subplots(figsize=(10, 8))

grafico = ax.bar(x = 'Churn', height=df_base.Churn, data = df_base)

for barra in grafico:
  # Pegando a altura(valor) e largura de cada barra
    altura = barra.get_height()
    largura = barra.get_width()

    '''
      Posição x(xpos) e y(ypos) de cada barra. Aqui temos três casos de y pos:
      1º - colocando o rótulo acima da barra
      2º - colocando o rótulo no meio da barra
      3º - colocando o rótulo na base da barra
      Para selecionar o desejado, é só descomentar o ypos que quer fazer uso.
    '''
    xpos = barra.get_x() + largura/2
    ypos = 1.01 * altura
    # ypos = barra.get_y() + altura/2
    # ypos = barra.get_y()

    # Adicionando o rótulo nas barras (x= coordenada x, y = coordenada y, s = texto desejado)
    ax.text(x = xpos, y = ypos, s = altura, ha='center', va='bottom', fontsize=14)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# grafico = df_genero.plot(kind = 'bar')
grafico = ax.bar(x = df_genero.columns, height=df_genero.loc['Yes'], data = df_genero)

for barra in grafico:
  # Pegando a altura(valor) e largura de cada barra
    altura = barra.get_height()
    largura = barra.get_width()

    '''
      Posição x(xpos) e y(ypos) de cada barra. Aqui temos três casos de y pos:
      1º - colocando o rótulo acima da barra
      2º - colocando o rótulo no meio da barra
      3º - colocando o rótulo na base da barra
      Para selecionar o desejado, é só descomentar o ypos que quer fazer uso.
    '''
    xpos = barra.get_x() + largura/2
    ypos = 1.01 * altura
    # ypos = barra.get_y() + altura/2
    # ypos = barra.get_y()

    # Adicionando o rótulo nas barras (x= coordenada x, y = coordenada y, s = texto desejado)
    ax.text(x = xpos, y = ypos, s = altura, ha='center', va='bottom', fontsize=14)

#### Se acima de 65 anos

In [ ]:
df_analise = pd.crosstab(df_base.Churn, df_base.mais_que_65_anos.astype(str))

fig, ax = plt.subplots(figsize=(10, 8))

grafico = ax.bar(x = df_analise.columns, height=df_analise.loc['Yes'], data = df_analise.loc['Yes'])

for barra in grafico:
  # Pegando a altura(valor) e largura de cada barra
    altura = barra.get_height()
    largura = barra.get_width()

    '''
      Posição x(xpos) e y(ypos) de cada barra. Aqui temos três casos de y pos:
      1º - colocando o rótulo acima da barra
      2º - colocando o rótulo no meio da barra
      3º - colocando o rótulo na base da barra
      Para selecionar o desejado, é só descomentar o ypos que quer fazer uso.
    '''
    xpos = barra.get_x() + largura/2
    ypos = 1.01 * altura
    # ypos = barra.get_y() + altura/2
    # ypos = barra.get_y()

    # Adicionando o rótulo nas barras (x= coordenada x, y = coordenada y, s = texto desejado)
    ax.text(x = xpos, y = ypos, s = altura, ha='center', va='bottom', fontsize=14)